#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   (quando tiver)


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao

        produtos.append(produto)
    return produtos


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns).text
            
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [ ]:
read = pd.read_excel('upseller_template.xlsx')
produtos = read[-1:]


for  index, produto in produtos.iterrows():
    # fornecedor = produto['Fornecedor']
    # codigo_produto = produto['Codigo Produto']
    # descricao = produto['Descrição']
    try:
    #     if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
    #         url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
    #     elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
    #         url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
    #     elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
    #         url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
    #     else:
    #         print('Indisponivel')
    #         continue
        
        url = 'https://construja.com.br/produto/101223/estrela-tapa-furo-ll-20mm'

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}
       
        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        comprimento = extrai.get("comprimento")

        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        categoria = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "SUB CATEGORIA"), "Não disponível")
        print(categoria)
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)




In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from env import fornecedores_noweb_scapping

# === 1. Carregar o arquivo ===
df = pd.read_excel('produtos.xlsx')

# === 2. Função para limpar e normalizar texto ===
def limpar_texto(txt):
    if not isinstance(txt, str):

        return ""
    # Remove acentos
    txt = unicodedata.normalize('NFD', txt)
    txt = txt.encode('ascii', 'ignore').decode('utf-8')
    # Converte para maiúsculas
    txt = txt.upper()
    # Remove caracteres especiais
    txt = re.sub(r'[^A-Z0-9 ]', '', txt)
    # Remove espaços duplicados
    txt = re.sub(r'\s+', ' ', txt).strip()
    return txt


# === 3. Lista de marcas fixas (forçadas a fazer parte da análise) ===
marcas_adicionais = [
    'SHIVA','BRASCOLA','FERTAK','NATICON','NOBRE','ADELBRAS','STA MARINA','FIRMEZA','OLIPLAS','STAM', 'R.C.A','CORONA','TOP FIO',
    'MAKITA','REXON','SOPRANO','FAST LUB','ACQUA','KIAN','SOUDAL','DEPLAST','TATU','GRENDHA', 'PANASONIC','PAULICEIA','MORIA',
    'ALLTAPE','PLASTILIT','TITANIUM','FERRARI','NEGREIRA','RADIAL','BLUMENAU','J.F METAIS','ART METAIS','LONAX', 'GHEL PLUS',
    'CHEMICOLOR','SANY','PROAQUA','BICROM','WAGO','TECHNA','METASUL','GARAPEIRA','ROCO','PRO FOAM','DURACELL','POLIBEL','AGELUX',
    'REMOX','RORATO','WAVES','LEGRAND','PAPAIZ','AGELUZ','IMPERIAL','HYDRA','KITUBO','NETTE','SANLIMP', 'EMA METAIS','M&M'
    'BEARS','ADTEX','GALO','PEGAFORTE','EMAVE','ELGIN','LUCONI','VEGA','MUNDIAL','DELTA', 'COBRECOM','PLASTBIG','FERE','SATA',
    'ALUMBRA', 'FIOLUX','PACETTA','ITAMBE','PEXCEL','EMAVA','KADESH','IBERE','GOLD','PERLEX','IMA','SAFETY','PIAL','PLASTIK',
    'OUROLUX', 'LED BEE','BELZER','PROTEG','NASTRO','FJ FERR', 'SAO RAFAEL', 'STILLUS','FLASH LIMP', 'ZUMPLAST','REDY','PRIMETECH',
    'SECALUX', 'PLASTIC','SUPER BONDER', 'ARTPLAS','FABER CASTELL','GUARANI','DEWALT','ALUREM','PERFIX ','STELL','FERROX','USAF',
    'INJET','CMK','BEARS','MINASUL','DEGOMASTER','SIENA'
]
marcas_adicionais = [limpar_texto(m) for m in marcas_adicionais]


# === 4. Dicionário de sinônimos / variações ===
mapa_variacoes = {
    'TRAMONTINA': ['TRAMON', 'TRAMONTINA', 'TRAMONTINA ELE', 'TRAMONTINA GARI', 'TRAMONTINA MULT', 'TRAM'],
    'JOMARCA': ['JOMARCA', 'JOMARC', 'JOMARCA JMC', 'JMC'],
    'NEW FIX': ['NEW FIX', 'NEWFIX', 'NEW-FIX', 'NEW F','PARAF CHIPBOARD CHATA PHS','PARAF SEX.ROS.SOBERBA'],
    'NOVA': ['NOVA TINTAS', 'NOVA'],
    'DRYKO': ['DRYKO', 'DRYKOPRIMER'],
    'SIKA': ['SIKATOP', 'SIKA'],
    'BRASFORT': ['BRASF', 'BRASFORT'],
    'ILUMI': ['ILUM', 'ILUMI'],
    'DENVER': ['DENVER','DENVERIMPER', 'DENVERLAJE','DENVERTEC','DENVERCRILL', 'DENVERFITA','DENVERFIX'],
    'THOMPSOM': ['THOMPSOM', 'THOMP'],
    'PEGAFORTE': ['PEGAFORTE', 'PEGA FORTE'],
    'CALHA FORTE': ['CALHA FORTE', 'CALHA FORT'],
    'VEDACIT': ['VEDACIT', 'BIANCO'],
    'LORENZETTI': ['LOREN','LORENZETTI','ORIG.L&C D','.PMR D.','.L&C '],
    'KADESH': ['BOT FLEX ELASTICO PRETA BI']
}


# === 5. Montar lista final de marcas conhecidas ===
marcas_planilha = df['Marca'].dropna().unique().tolist()
marcas_planilha.remove('5+')
marcas_planilha = [limpar_texto(m) for m in marcas_planilha if isinstance(m, str) and m.strip()]

# Combinar todas
marcas = list(set(marcas_planilha + marcas_adicionais))

# Adicionar também as variações do mapa
for marca_principal, variacoes in mapa_variacoes.items():
    marcas.append(marca_principal)
    marcas.extend([limpar_texto(v) for v in variacoes])

marcas = list(set(marcas))


# === 7. Função para detectar marca na descrição ===
def detectar_marca(descricao):
    desc_limpa = limpar_texto(descricao)

    # Verifica variações conhecidas primeiro
    for marca_padrao, variacoes in mapa_variacoes.items():
        for v in variacoes:
            if limpar_texto(v) in desc_limpa:
                return marca_padrao

    # Caso não encontre nas variações, busca direta na lista geral
    for marca in marcas:
        if marca and marca in desc_limpa:
            return marca

    return np.nan

df_filtrado = df.copy()

# === 8. Detectar e preencher marcas ausentes ===
df_filtrado['Marca Detectada'] = df_filtrado['Descrição'].apply(detectar_marca)

# Preenche apenas onde está vazio
df_filtrado['Marca'] = df_filtrado['Marca'].fillna(df_filtrado['Marca Detectada'])

df_filtrado['Marca'] = df_filtrado['Marca'].fillna('GENÉRICO')
# Atualiza no dataframe principal
df.update(df_filtrado)


# === 9. Salvar resultado ===
df.to_excel('produtos_com_marcas.xlsx', index=False)

# === 10. Relatório simples ===
print("✅ Processamento concluído!")
print(f"Total de produtos analisados: {len(df_filtrado)}")
print(f"Marcas detectadas automaticamente: {df_filtrado['Marca Detectada'].notna().sum()}")

# Top 10 marcas mais detectadas
print("\nTop marcas detectadas:")
print(df_filtrado['Marca Detectada'].value_counts().head(10))


In [ ]:
import pandas as pd
import re
import unicodedata

df = pd.read_excel('pai_filho_variantes.xlsx')

# --- Função para normalizar ---
def normalizar(texto):
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'\s+', ' ', texto)
    return texto

df["chave_norm"] = df["chave"].apply(normalizar)

# --- Função para encontrar a base e variação dentro de um grupo ---
def extrair_base_variacao(grupo_df):
    textos = grupo_df["chave_norm"].tolist()
    palavras = [set(t.split()) for t in textos]

    comuns = set.intersection(*palavras) 
    bases, variacoes = [], []
    for texto in textos:
        palavras_texto = texto.split()
        variacao = [p for p in palavras_texto if p not in comuns]
        base = [p for p in palavras_texto if p in comuns]
        bases.append(" ".join(base))
        variacoes.append(" ".join(variacao))
    
    grupo_df["base"] = bases
    grupo_df["variacao"] = variacoes
    return grupo_df

# --- Aplica por grupo ---
df_resultado = (
    df.groupby("ID_Variacao", group_keys=False)
      .apply(lambda g: extrair_base_variacao(g.copy()))
      .reset_index(drop=True)
)

df_resultado.to_excel('base_nome.xlsx', index=False)

In [31]:
import pandas as pd
from rapidfuzz import fuzz
import unicodedata
from tqdm import tqdm
from difflib import SequenceMatcher
from logger import logger

# --- Parâmetro ajustável ---
LIMIAR = 89          # similaridade entre chaves (fuzz)
SENSIBILIDADE = 0.4  # sensibilidade da parte comum (0 a 1)

# --- Função para normalizar textos ---
def normalizar(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).upper().strip()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    return texto

# --- Carrega o Excel ---
df = pd.read_excel("pai_filho_variantes.xlsx")

df = df[:20]

# --- Cria chave composta ---
df["Chave"] = (
    df["Descrição"].apply(normalizar) + " " +
    df["Categoria"].apply(normalizar)
)

# --- Inicializa controle ---
df["ID_Variacao"] = None
df["Tipo"] = None
df["SKU_Pai"] = None
grupo_id = 1
usados = set()

# --- Agrupamento ---
print("🔄 Agrupando variações...")

for i, linha in tqdm(df.iterrows(), total=len(df)):
    if i in usados:
        continue

    chave_ref = linha["Chave"]
    similares = df.index[df["Chave"].apply(lambda x: fuzz.token_sort_ratio(chave_ref, x) >= LIMIAR)].tolist()

    sku_pai = linha.get("Sku", linha.get("Código", i))
    df.at[i, "Tipo"] = "PAI"
    df.at[i, "SKU_Pai"] = sku_pai
    df.at[i, "ID_Variacao"] = grupo_id

    for idx in similares:
        if idx not in usados:
            if idx != i:
                df.at[idx, "Tipo"] = "FILHO"
                df.at[idx, "SKU_Pai"] = sku_pai
            df.at[idx, "ID_Variacao"] = grupo_id
            usados.add(idx)

    grupo_id += 1


# --- Ordena ---
df = df.sort_values(by=["ID_Variacao", "Tipo"], ascending=[True, True]).reset_index(drop=True)

# --- Função com controle de sensibilidade ---
def parte_comum(strings, sensibilidade=SENSIBILIDADE):
    if not strings:
        return ""
    base = strings[0]
    for s in strings[1:]:
        seq = SequenceMatcher(None, base, s)
        match = seq.find_longest_match(0, len(base), 0, len(s))
        comum = base[match.a: match.a + match.size]

        tamanho_medio = (len(base) + len(s)) / 2
        proporcao = len(comum) / tamanho_medio

        # Mantém só se atingir a sensibilidade mínima
        if proporcao >= sensibilidade:
            base = comum
        else:
            base = comum[:int(len(comum) * sensibilidade)]

    return base.strip()

# --- Cria colunas base e variante ---
print("🧩 Gerando colunas base e variante...")
df["Base"] = ""
df["Variante"] = ""

print(df[['ID_Variacao','Chave']])

for gid, grupo in df.groupby("ID_Variacao"):
    chaves = grupo["Chave"].tolist()
    comum = parte_comum(chaves, SENSIBILIDADE)

    for idx, linha in grupo.iterrows():
        texto = linha["Chave"]
        variante = texto.replace(comum, "").strip()
        df.at[idx, "Base"] = comum
        df.at[idx, "Variante"] = variante


# display(df[['Chave','Base','Variante']])

# # --- Salva ---
# arquivo_saida = "base_nome.xlsx"
# df.to_excel(arquivo_saida, index=False)

# print("✅ Agrupamento concluído com sucesso!")
# print(f"📂 Arquivo salvo como: {arquivo_saida}")
# print(f"📦 Total de grupos de variações criados: {grupo_id - 1}")


🔄 Agrupando variações...


100%|██████████| 20/20 [00:00<00:00, 1403.86it/s]

🧩 Gerando colunas base e variante...
   ID_Variacao                                              Chave
0            1         FITA CREPE 48X50 USO GERAL MASK FITA CREPE
1            1         FITA CREPE 24X50 USO GERAL MASK FITA CREPE
2            2   VALV RETENCAO ESG 100/75 ESTRELA CONEXOES ESGOTO
3            2    VALV RETENCAO ESG 50/40 ESTRELA CONEXOES ESGOTO
4            3  VELA FILTRO GRAVIDADE V/C CRISTALINA DIVERSOS ...
5            4  CHAVE PHILLIPS CRV TOCO C/PI 1/4X11/2 FAMASTIL...
6            5  CHAVE FENDA TOCO CRV C/PI 3/16X11/2 FOX/FAMAST...
7            6  LAMP LED ALTA POT 30W-2400LM 6500K GALAXY LAMP...
8            6  LAMP LED ALTA POT 40W-3200LM 6500K GALAXY LAMP...
9            6  LAMP LED ALTA POT 50W-4000LM 6500K GALAXY LAMP...
10           6  LAMP LED ALTA POT 20W-1600LM 6500K GALAXY LAMP...
11           7  LAMP LED ALTA POT 50W-4000LM 6500K FOXLUX LAMP...
12           8  LAMP LED ALTA POT 50W-4000LM 6500K AVANT LAMPADAS
13           9         CX DESC S/E 6/9L

In [18]:

from rapidfuzz import fuzz, process

a = 'CHAVE PHILLIPS CRV TOCO C/PI 1/4X11/2 FAMASTIL FERRAMENTAS MANUAIS E ACESSORIOS'
b= 'CHAVE FENDA TOCO CRV C/PI 3/16X11/2 FOX/FAMASTIL FERRAMENTAS MANUAIS E ACESSORIOS'



# print(f"Similarity score: {fuzz.token_sort_ratio(a, b)}")

# print(process.extract(collection, scorer=fuzz.ratio))


set_a = set(a.split())
set_b = set(b.split())

semelhantes = set_a.intersection(set_b)
diferentes_a = set_a - set_b
diferentes_b = set_b - set_a

print("Iguais:", semelhantes)
print("Só em A:", diferentes_a)
print("Só em B:", diferentes_b)


Iguais: {'TOCO', 'E', 'FERRAMENTAS', 'ACESSORIOS', 'CRV', 'C/PI', 'CHAVE', 'MANUAIS'}
Só em A: {'1/4X11/2', 'PHILLIPS', 'FAMASTIL'}
Só em B: {'FENDA', 'FOX/FAMASTIL', '3/16X11/2'}
